In [1]:
from imutils.video import VideoStream
from imutils import face_utils
from pygame import mixer
mixer.init()

import imutils
import numpy as np
import time
import dlib
import cv2

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#計算眼睛的寬高比
def eyesRatio(eye):
    hight1 = np.linalg.norm(eye[1] - eye[5])
    hight2 = np.linalg.norm(eye[2] - eye[4])
    width = np.linalg.norm(eye[2] - eye[4])
    return (hight1+hight2) / (2.0*width)

In [3]:
#眼睛寬高比的臨界值，由前30張圖像平均而得
eyesRatioLimit=0  
#數據採集的計數，採集30次然後取平均值
collectCount=0  
#用於數據採集的求和
collectSum=0  
#是否開始檢測
startCheck=False  
#統計"閉眼"的次數
eyesCloseCount=0  
#統計"閉眼"的次數
eyesOpenCount=0 

In [ ]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("Model/shape_predictor_68_face_landmarks.dat")

#左右眼索引
(left_Start,left_End)=face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_Start,right_End)=face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

#開啟攝影機，延遲2秒鐘
vsThread=VideoStream(src=0).start()
time.sleep(2.0)

#循環檢測
while True:
    frame = vsThread.read()
    frame = imutils.resize(frame, width=720)
    faces = detector(frame, 0)  
    for face in faces:
        shape = predictor(frame, face)  
        shape = face_utils.shape_to_np(shape)  
        
        #左右眼
        leftEye = shape[left_Start:left_End]
        rightEye = shape[right_Start:right_End]
        leftEyesVal = eyesRatio(leftEye)
        rightEyesVal = eyesRatio(rightEye)
        
        #凸殼
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        #繪製輪廓
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        #取兩隻眼長寬比的的平均值
        eyeRatioVal = (leftEyesVal + rightEyesVal) / 2.0 
        
        #計算30次平均值
        if collectCount<30:
            collectCount+=1
            collectSum+=eyeRatioVal
            startCheck=False
        else:
            if not startCheck:
                eyesRatioLimit=collectSum/(1.0*30)
            startCheck=True

        #閉眼次數超過指定次則認為已經睡著
        if startCheck:
            
            #如果眼睛長寬比大於之前檢測到的臨界值，則計數
            if eyeRatioVal < eyesRatioLimit:  
                eyesCloseCount += 1
                eyesOpenCount = 0
                
                #次數超過設定值
                if eyesCloseCount >= 30:  
                    cv2.putText(frame, "SLEEP!!!", (580, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                    #發出聲音
                    mixer.music.load('Image_test/bb.wav')
                    mixer.music.play()  
            else: 
                eyesOpenCount += 1
                if eyesOpenCount > 2:
                    eyesCloseCount = 0
            
            #眼睛寬高比
            cv2.putText(frame, "EYES_RATIO: {:.2f}".format(eyeRatioVal), (20, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 160, 0), 2)
            
            #閉眼次數
            cv2.putText(frame,"EYES_COLSE: {}".format(eyesCloseCount),(320,30),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,160,0),2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(10) == 27:  break

cv2.destroyAllWindows()
vsThread.stop()